In [1]:
!pip install ortools

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random

## RANDOM DATA

In [3]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [ [ 0, 14, 25, 20, 22,  0],
                                [14,  0, 11, 14, 36, 31],
                                [25, 11,  0, 15, 46, 42],
                                [20, 14, 15,  0, 36, 40],
                                [22, 36, 46, 36,  0, 22],
                                [ 0, 31, 42, 40, 22,  0]]
    data['demands'] = [0, 1, 5, 8, 5, 5]
    data['vehicle_capacities'] = [10, 10, 10]
    data['num_vehicles'] = 3
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def testGoogleOR():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


testGoogleOR()

Route for vehicle 0:
 0 Load(0) ->  3 Load(8) ->  0 Load(8)
Distance of the route: 40m
Load of the route: 8

Route for vehicle 1:
 0 Load(0) ->  1 Load(1) ->  2 Load(6) ->  0 Load(6)
Distance of the route: 50m
Load of the route: 6

Route for vehicle 2:
 0 Load(0) ->  5 Load(5) ->  4 Load(10) ->  0 Load(10)
Distance of the route: 44m
Load of the route: 10

Total distance of all routes: 134m
Total load of all routes: 24


## TESTING PRE-DEFINED DATA N-15

In [4]:
# Data Source: 
numberOfCustomers = 15
capacityOfVehicle = 35
numberOfVehicles = 8
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C #+ [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {0:0,1:19,2:30,3:16,4:23,5:11,6:31,7:15,8:28,9:8,10:8,11:7,12:14,13:6,14:19,15:11}
# demandOfCustomers[0] = 0
# demandOfCustomers[numberOfCustomers+1] = 0

xCoordinates = [30,37,49,52,31,52,42,52,57,62,42,27,43,58,58,37]
yCoordinates = [40,52,49,64,62,33,41,41,58,42,57,68,67,48,27,69]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 0
costMatrix

array([[ 0., 13., 21., 32., 22., 23., 12., 22., 32., 32., 20., 28., 29.,
        29., 30.,  0.],
       [13.,  0., 12., 19., 11., 24., 12., 18., 20., 26.,  7., 18., 16.,
        21., 32., 17.],
       [21., 12.,  0., 15., 22., 16., 10.,  8., 12., 14., 10., 29., 18.,
         9., 23., 23.],
       [32., 19., 15.,  0., 21., 31., 25., 23.,  7., 24., 12., 25.,  9.,
        17., 37., 15.],
       [22., 11., 22., 21.,  0., 35., 23., 29., 26., 36., 12.,  7., 13.,
        30., 44.,  9.],
       [23., 24., 16., 31., 35.,  0., 12.,  8., 25., 13., 26., 43., 35.,
        16.,  8., 39.],
       [12., 12., 10., 25., 23., 12.,  0., 10., 22., 20., 16., 30., 26.,
        17., 21., 28.],
       [22., 18.,  8., 23., 29.,  8., 10.,  0., 17., 10., 18., 36., 27.,
         9., 15., 31.],
       [32., 20., 12.,  7., 26., 25., 22., 17.,  0., 16., 15., 31., 16.,
        10., 31., 22.],
       [32., 26., 14., 24., 36., 13., 20., 10., 16.,  0., 25., 43., 31.,
         7., 15., 36.],
       [20.,  7., 10., 12., 12

In [5]:
"""Vehicles Routing Problem (VRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = costMatrix
    data['demands'] = [0,19,30,16,23,11,31,15,28,8,8,7,14,6,19,11]
    data['vehicle_capacities'] = [35, 35, 35, 35, 35, 35, 35, 35]
    data['num_vehicles'] = 8
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 Load(0) ->  8 Load(28) ->  0 Load(28)
Distance of the route: 64m
Load of the route: 28

Route for vehicle 1:
 0 Load(0) ->  1 Load(19) ->  3 Load(35) ->  0 Load(35)
Distance of the route: 64m
Load of the route: 35

Route for vehicle 2:
 0 Load(0) ->  11 Load(7) ->  4 Load(30) ->  0 Load(30)
Distance of the route: 57m
Load of the route: 30

Route for vehicle 3:
 0 Load(0) ->  14 Load(19) ->  5 Load(30) ->  0 Load(30)
Distance of the route: 61m
Load of the route: 30

Route for vehicle 4:
 0 Load(0) ->  2 Load(30) ->  0 Load(30)
Distance of the route: 42m
Load of the route: 30

Route for vehicle 5:
 0 Load(0) ->  13 Load(6) ->  9 Load(14) ->  7 Load(29) ->  0 Load(29)
Distance of the route: 68m
Load of the route: 29

Route for vehicle 6:
 0 Load(0) ->  6 Load(31) ->  0 Load(31)
Distance of the route: 24m
Load of the route: 31

Route for vehicle 7:
 0 Load(0) ->  15 Load(11) ->  12 Load(25) ->  10 Load(33) ->  0 Load(33)
Distance of the route: 36m
Load of the route:

## TESTING PRE-DEFINED DATA N-31

In [0]:
numberOfCustomers = 31
capacityOfVehicle = 100
numberOfVehicles = 5
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C #+ [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = [0,19,21,6,19,7,12,16,6,16,8,14,21,16,3,22,18,19,1,24,8,12,4,8,24,24,2,20,15,2,14,9]
# demandOfCustomers[0] = 0
# demandOfCustomers[numberOfCustomers+1] = 0

In [7]:
xCoordinates = [82,96,50,49,13,29,58,84,14,2,3,5,98,84,61,1,88,91,19,93,50,98,5,42,61,9,80,57,23,20,85,98,82]
yCoordinates = [76,44,5,8,7,89,30,39,24,39,82,10,52,25,59,65,51,2,32,3,93,14,42,9,62,97,55,69,15,70,60,5,76]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 0
costMatrix

array([[  0.,  34.,  77., ...,  62.,  16.,   0.],
       [ 34.,   0.,  60., ...,  80.,  19.,  39.],
       [ 77.,  60.,   0., ...,  71.,  65.,  48.],
       ...,
       [ 62.,  80.,  71., ...,   0.,  65., 101.],
       [ 16.,  19.,  65., ...,  65.,   0.,  56.],
       [  0.,  39.,  48., ..., 101.,  56.,   0.]])

In [8]:
"""Vehicles Routing Problem (VRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = costMatrix
    data['demands'] = demandOfCustomers
    data['vehicle_capacities'] = [100, 100, 100, 100, 100]
    data['num_vehicles'] = 5
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 Load(0) ->  27 Load(20) ->  24 Load(44) ->  0 Load(44)
Distance of the route: 58m
Load of the route: 44

Route for vehicle 1:
 0 Load(0) ->  6 Load(12) ->  3 Load(18) ->  2 Load(39) ->  23 Load(47) ->  4 Load(66) ->  11 Load(80) ->  28 Load(95) ->  14 Load(98) ->  0 Load(98)
Distance of the route: 225m
Load of the route: 98

Route for vehicle 2:
 0 Load(0) ->  20 Load(8) ->  5 Load(15) ->  25 Load(39) ->  10 Load(47) ->  29 Load(49) ->  15 Load(71) ->  22 Load(75) ->  9 Load(91) ->  8 Load(97) ->  18 Load(98) ->  0 Load(98)
Distance of the route: 264m
Load of the route: 98

Route for vehicle 3:
 0 Load(0) ->  12 Load(21) ->  1 Load(40) ->  16 Load(58) ->  30 Load(72) ->  0 Load(72)
Distance of the route: 71m
Load of the route: 72

Route for vehicle 4:
 0 Load(0) ->  31 Load(9) ->  19 Load(33) ->  17 Load(52) ->  21 Load(64) ->  13 Load(80) ->  7 Load(96) ->  26 Load(98) ->  0 Load(98)
Distance of the route: 88m
Load of the route: 98

Total distance of all routes